# Different TTS engines comparisons

We tried two different TTS (text-to-speech) engines:

## pyttsx3
It works offline, uses the voices in your local system for TTS. The voices don't seem completely natural

`pip install pyttsx3`

[GitHub](https://github.com/nateshmbhat/pyttsx3)

## gTTS
It uses Google TTS and gives very natural voices. It can be saved directly to file and unlimited length. It needs internet connection.

`pip install gtts`

[GitHub](https://github.com/pndurette/gTTS)


All examples run with Python version: 3.11.3

# Further reading
- https://www.geeksforgeeks.org/convert-text-speech-python/
- https://pyttsx3.readthedocs.io/en/latest/engine.html#examples
- https://gtts.readthedocs.io/en/latest/module.html#localized-accents
- https://gtts.readthedocs.io/en/latest/module.html#examples


# Other options to explore
I wanted to use Tik Tok voices, this can be an interesting choice: https://rapidapi.com/dalamates/api/tts-tiktok

In [32]:
import pyttsx3

engine = pyttsx3.init()

engine.setProperty("rate", 100)
engine.setProperty("volume", 1.0)

engine.say("The red house.")
engine.say("A big dog.")

engine.runAndWait()

In [33]:
# Different voices
import pyttsx3

engine = pyttsx3.init()

voices = engine.getProperty("voices")

for voice in voices:
  print(voice)

<Voice id=HKEY_LOCAL_MACHINE\SOFTWARE\Microsoft\Speech\Voices\Tokens\TTS_MS_EN-GB_HAZEL_11.0
          name=Microsoft Hazel Desktop - English (Great Britain)
          languages=[]
          gender=None
          age=None>
<Voice id=HKEY_LOCAL_MACHINE\SOFTWARE\Microsoft\Speech\Voices\Tokens\TTS_MS_EN-US_ZIRA_11.0
          name=Microsoft Zira Desktop - English (United States)
          languages=[]
          gender=None
          age=None>
<Voice id=HKEY_LOCAL_MACHINE\SOFTWARE\Microsoft\Speech\Voices\Tokens\TTS_MS_ES-ES_HELENA_11.0
          name=Microsoft Helena Desktop - Spanish (Spain)
          languages=[]
          gender=None
          age=None>


In my computer I get these results:
```ini
<Voice id=HKEY_LOCAL_MACHINE\SOFTWARE\Microsoft\Speech\Voices\Tokens\TTS_MS_EN-GB_HAZEL_11.0
          name=Microsoft Hazel Desktop - English (Great Britain)
          languages=[]
          gender=None
          age=None>
<Voice id=HKEY_LOCAL_MACHINE\SOFTWARE\Microsoft\Speech\Voices\Tokens\TTS_MS_EN-US_ZIRA_11.0
          name=Microsoft Zira Desktop - English (United States)
          languages=[]
          gender=None
          age=None>
<Voice id=HKEY_LOCAL_MACHINE\SOFTWARE\Microsoft\Speech\Voices\Tokens\TTS_MS_ES-ES_HELENA_11.0
          name=Microsoft Helena Desktop - Spanish (Spain)
          languages=[]
          gender=None
          age=None>
```

In [35]:
import pyttsx3

engine = pyttsx3.init()

engine.setProperty("rate", 100)
engine.setProperty("volume", 1.0)

voices = engine.getProperty("voices")

for voice in voices:
  print(f"Voice: {voice.name}")
  engine.setProperty('voice', voice.id)
  engine.say("The red house.")
  engine.say("A big dog.")

engine.runAndWait()

Voice: Microsoft Hazel Desktop - English (Great Britain)
Voice: Microsoft Zira Desktop - English (United States)
Voice: Microsoft Helena Desktop - Spanish (Spain)


In [ ]:
from gtts import gTTS
import os

text = "The red house. A big dog."
tts = gTTS(text=text, lang="en", tld="us", slow=True) # American accent

tts.save("test.mp3")

os.system("start test.mp3")  # Play in Windows

0

In [42]:
from gtts import gTTS
import os

text = "A red house. A big dog."
tts = gTTS(text=text, lang="en", tld="co.uk", slow=True) # British accent

tts.save("test.mp3")

os.system("start test.mp3")  # Play in Windows

0